In [3]:
from datetime import datetime
import pytz

def fecha_hora_lima():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y%m%d-%H%M%S')

def fecha_hora_lima_db():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y-%m-%d %H:%M:%S')

fecha_hora_para_archivos = fecha_hora_lima()
fecha_hora_para_base_de_datos = fecha_hora_lima_db()

In [4]:
import pandas as pd
import requests
import json

# URL del JSON
url = 'https://www.wong.pe/api/catalog_system/pub/category/tree/3'

# Solicitar el contenido JSON desde la URL
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta en un objeto JSON
    file_name = f'categories_{fecha_hora_para_archivos}.json'
    
    # Guardar el JSON en un archivo
    with open(file_name, 'w') as file:
        json.dump(data, file, indent=4)

    print('El archivo JSON se ha guardado exitosamente.')
    with open(file_name, 'r') as file:
        data = json.load(file)
    print(len(data)) 
else:
    print(f'Error al obtener el JSON. Código de estado: {response.status_code}')  

El archivo JSON se ha guardado exitosamente.
27


In [5]:
# file_path = 'wong_categories.json'
# 
# with open(file_path, 'r') as file:
#     data = json.load(file)
#     print(json.dumps(data, indent=4))

In [6]:
columns = ['id', 'name', 'hasChildren', 'url', 'NumChildren', 'Title', 'MetaTagDescription', 'parent_id']
df = pd.DataFrame(columns=columns)
df

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [7]:
for item in data:
  if item['hasChildren']:
    item['parent_id'] = ''
    item['NumChildren'] = len(item['children'])

    df.loc[len(df)] = item

    parent_id = item['id']

    for subitem in item['children']:
      subitem['parent_id'] = parent_id
      sub_parent_id = subitem['id']
      subitem['NumChildren'] = len(subitem['children'])

      df.loc[len(df)] = subitem

      if len(subitem['children']) > 0:
        for subsubitem in subitem['children']:
          subsubitem['parent_id'] = sub_parent_id
          subsubitem['NumChildren'] = len(subsubitem['children'])
          df.loc[len(df)] = subsubitem

In [8]:
df.query("id == 952")

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [9]:
file_name_categories_csv = f'wong_categories_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_categories_csv, index=False)

In [10]:
cats = df.query("parent_id == ''")['id'].to_list()
print(cats)

[800, 1200, 1400, 1700, 1800, 1900, 2000, 2100, 2200, 1000144, 1000003, 1000131, 1000143, 1000069, 1001074, 1001253, 1001327, 2301, 1001374, 1001402, 1001436, 1001467, 1001470, 1001499, 1001673, 1001701]


In [11]:
import time

products = []

for cat in cats:
    start = 0
    end = 49
    
    while True:
        time.sleep(1)
        url = f"https://www.wong.pe/api/catalog_system/pub/products/search?fq=C:{cat}&_from={start}&_to={end}&O=OrderByPriceDESC"
        
        response = requests.get(url)
        
        if response.status_code == 200 or response.status_code == 206:
            data = response.json()
            if not data:
                break
                 
            products.extend(data)
            start += 50
            end += 50
        else:
            break
    
    print(f"status_code = {response.status_code}")
    print(f"cat = {cat}, start = {start}, end = {end}")

 
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.json'

with open(file_name_productos_csv, 'w') as file:
    json.dump(products, file, indent=4)

status_code = 200
cat = 800, start = 600, end = 649
status_code = 500
cat = 1200, start = 0, end = 49
status_code = 200
cat = 1400, start = 550, end = 599
status_code = 500
cat = 1700, start = 2100, end = 2149
status_code = 200
cat = 1800, start = 650, end = 699
status_code = 200
cat = 1900, start = 1200, end = 1249
status_code = 200
cat = 2000, start = 1000, end = 1049
status_code = 500
cat = 2100, start = 1400, end = 1449
status_code = 400
cat = 2200, start = 2550, end = 2599
status_code = 200
cat = 1000144, start = 300, end = 349
status_code = 200
cat = 1000003, start = 350, end = 399
status_code = 400
cat = 1000131, start = 2550, end = 2599
status_code = 200
cat = 1000143, start = 1250, end = 1299
status_code = 200
cat = 1000069, start = 600, end = 649
status_code = 200
cat = 1001074, start = 2250, end = 2299
status_code = 200
cat = 1001253, start = 1450, end = 1499
status_code = 200
cat = 1001327, start = 750, end = 799
status_code = 200
cat = 2301, start = 2150, end = 2199
status

In [ ]:
print(len(products))

In [13]:
columns = ['productId', 'productName', 'brand', 'brandId', 'brandImageUrl','categoryId',
           'productTitle', 'metaTagDescription', 'categories', 'categoriesIds', 'link', 'Vendido por',
           'Descripción del producto', 'Presentación', 'Presentación/Empaque', 'Pack', 'Porción Sugerida',
           'Azúcares Por Porción', 'Carbohidratos Por Porción', 'Descripción Nutricional', 'Grasas Por Porción',
           'Advertencias de consumo', 'itemId', 'ean', 'imageUrl', 'Value', 'IsAvailable', 'AvailableQuantity',
           'Price', 'ListPrice', 'PriceWithoutDiscount', 'PriceValidUntil', 'Created']

df = pd.DataFrame(columns=columns)
df

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created


In [14]:
for product in products:
  print(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'])
  break

279.0
2100-01-01T04:59:59Z
279.0
279.0


In [15]:
print(json.dumps(products[3099], indent=4))

{
    "productId": "551921",
    "productName": "Dip de Pimiento Piquillo con Queso Frasco 200 g",
    "brand": "Casa Verde",
    "brandId": 9935,
    "brandImageUrl": null,
    "linkText": "dip-de-pimiento-piquillo-con-queso-frasco-200-g-2-2",
    "productReference": "737875001",
    "productReferenceCode": "737875001",
    "categoryId": "1736",
    "productTitle": null,
    "metaTagDescription": null,
    "releaseDate": "2024-09-27T00:00:00Z",
    "clusterHighlights": {
        "6320": "15% dscto en todo Conservas",
        "6516": "15% dscto en conservas",
        "7035": "TC Abarrotes y Perecibles",
        "7083": "Agenda Ahorro Wong Lunes",
        "7086": "Agenda Ahorro Metro Lunes",
        "10741": "Dinamica Paneton Metro",
        "10899": "Delivery Gratis T121 Playas",
        "10901": "Delivery Gratis T116 Playas",
        "11044": "Abarrotes Wong",
        "11069": "Supermercado Metro",
        "11746": "Abarrotes wong audiencia promo paneton",
        "11747": "Abarrotes 

In [16]:
# print(json.dumps(products, indent=4))

In [17]:
def print_value(value, key):
    if value:
        print(f"{key} = {value}")
    else:
        print(f"{key} = None")

In [18]:
from bs4 import BeautifulSoup

def clean_html(html_code):
    if html_code is None:
        return ''
    if isinstance(html_code, list):
        html_code = ' '.join(html_code)
    soup = BeautifulSoup(html_code, 'html.parser')
    return soup.get_text()

In [19]:
def get_product_data(product, fecha_db):
    product['Created'] = fecha_db
    new_product = {}
    new_product['productId'] = product.get('productId')
    new_product['productName'] = product.get('productName')
    new_product['brand'] = product.get('brand')
    new_product['brandId'] = product.get('brandId')
    new_product['brandImageUrl'] = product.get('brandImageUrl')
    new_product['categoryId'] = product.get('categoryId')
    new_product['productTitle'] = product.get('productTitle')
    new_product['metaTagDescription'] = product.get('metaTagDescription')
    new_product['categories'] = product.get('categories')
    new_product['categoriesIds'] = product.get('categoriesIds')
    new_product['link'] = product.get('link')
    new_product['Vendido por'] = product.get('Vendido por')
    new_product['Descripción del producto'] = clean_html(product.get('Descripción del producto'))
    new_product['Presentación'] = product.get('Presentación')
    new_product['Presentación/Empaque'] = product.get('Presentación/Empaque')
    new_product['Pack'] = product.get('Pack')
    new_product['Porción Sugerida'] = product.get('Porción Sugerida')
    new_product['Azúcares Por Porción'] = product.get('Azúcares Por Porción')
    new_product['Carbohidratos Por Porción'] = product.get('Carbohidratos Por Porción')
    new_product['Descripción Nutricional'] = product.get('Descripción Nutricional')
    new_product['Grasas Por Porción'] = product.get('Grasas Por Porción')
    new_product['Advertencias de consumo'] = product.get('Advertencias de consumo')
    
    if product['items']:
        new_product['itemId'] = product['items'][0].get('itemId', None)
        new_product['ean'] = product['items'][0].get('ean', None)
        if product['items'][0]['images']:
            new_product['imageUrl'] = product['items'][0]['images'][0].get('imageUrl', None)
            if product['items'][0]['sellers']:
                commertial_offer = product['items'][0]['sellers'][0].get('commertialOffer', {})
                if commertial_offer.get('Installments'):
                    new_product['Value'] = commertial_offer['Installments'][0].get('Value', None)
                new_product['IsAvailable'] = commertial_offer.get('IsAvailable', None)
                new_product['AvailableQuantity'] = commertial_offer.get('AvailableQuantity', None)
                new_product['Price'] = commertial_offer.get('Price', None)
                new_product['ListPrice'] = commertial_offer.get('ListPrice', None)
                new_product['PriceWithoutDiscount'] = commertial_offer.get('PriceWithoutDiscount', None)
                new_product['PriceValidUntil'] = commertial_offer.get('PriceValidUntil', None)

    new_product['Created'] = product['Created']
    
    return new_product

In [20]:
# Para validar que los datos se están extrayendo correctamente

i = 0
for product in products:
    i += 1
    product['Created'] = fecha_hora_lima_db()
    print_value(product.get('productId'), 'productId')
    print_value(product.get('productName'), 'productName')
    print_value(product.get('brand'), 'brand')
    print_value(product.get('brandId'), 'brandId')
    print_value(product.get('brandImageUrl'), 'brandImageUrl')
    print_value(product.get('categoryId'), 'categoryId')
    print_value(product.get('productTitle'), 'productTitle')
    print_value(product.get('metaTagDescription'), 'metaTagDescription')
    print_value(product.get('categories'), 'categories')
    print_value(product.get('categoriesIds'), 'categoriesIds')
    print_value(product.get('link'), 'link')
    print_value(product.get('Vendido por'), 'Vendido por')
    print_value(clean_html(product.get('Descripción del producto')), 'Descripción del producto')
    print_value(product.get('Presentación'), 'Presentación')
    print_value(product.get('Presentación/Empaque'), 'Presentación/Empaque')
    print_value(product.get('Pack'), 'Pack')
    print_value(product.get('Porción Sugerida'), 'Porción Sugerida')
    print_value(product.get('Azúcares Por Porción'), 'Azúcares Por Porción')
    print_value(product.get('Carbohidratos Por Porción'), 'Carbohidratos Por Porción')
    print_value(product.get('Descripción Nutricional'), 'Descripción Nutricional')
    print_value(product.get('Grasas Por Porción'), 'Grasas Por Porción')
    print_value(product.get('Advertencias de consumo'), 'Advertencias de consumo')
    print_value(product['items'][0]['itemId'], 'itemId')
    print_value(product['items'][0]['ean'], 'ean')
    print_value(product['items'][0]['images'][0]['imageUrl'], 'imageUrl')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'], 'Value')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['IsAvailable'], 'IsAvailable')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['AvailableQuantity'], 'AvailableQuantity')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Price'], 'Price')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'], 'ListPrice')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'], 'PriceWithoutDiscount')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'], 'PriceValidUntil')
    print_value(product['Created'], 'Created')
    
    if i == 3:
        break

productId = 939222
productName = Fruta Deshidratada Manzana, Fresa y Melón Natifrut x kg
brand = Genérico
brandId = 9619
brandImageUrl = None
categoryId = 1001317
productTitle = Fruta Deshidratada Manzana, Fresa y Melón Natifrut x kg
metaTagDescription = ¡Encuentra aquí la mejor variedad de Frutos Secos a Granel de la marca Natifrut!
categories = ['/Frutas y Verduras/Frutas/Frutos Secos/', '/Frutas y Verduras/Frutas/', '/Frutas y Verduras/']
categoriesIds = ['/800/801/1001317/', '/800/801/', '/800/']
link = https://www.wong.pe/fruta-deshidratada-manzana-fresa-y-melon-natifrut-x-kg-771048-2/p
Vendido por = None
Descripción del producto = None
Presentación = None
Presentación/Empaque = None
Pack = None
Porción Sugerida = None
Azúcares Por Porción = None
Carbohidratos Por Porción = None
Descripción Nutricional = None
Grasas Por Porción = None
Advertencias de consumo = None
itemId = 39179921
ean = 22415
imageUrl = https://wongfood.vteximg.com.br/arquivos/ids/509042/Mix-Manzana-Fresa-Mel-n-

In [ ]:
for product in products:
  new_product = get_product_data(product, fecha_hora_para_base_de_datos)
  df.loc[len(df)] = new_product

In [ ]:
df.sample(n=5)

In [ ]:
df[['productId', 'productName', 'brand', 'categoryId', 'link', 'Price', 'AvailableQuantity']].sample(n=8)

In [ ]:
df.info()

In [ ]:
df['product_uri_json'] = df['productId'].apply(lambda x: f'https://www.wong.pe/api/catalog_system/pub/products/search?fq=productId:{x}')

In [ ]:
df['product_uri_json'].sample(n=5)

In [ ]:
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_productos_csv, index=False)

In [ ]:
df.iloc[26955]